<a href="https://colab.research.google.com/github/szezlong/gene-classification/blob/main/Gene_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Klasyfikacja choroby nowotworowej za pomocą analizy ekspresji genów
Źródło zbioru danych: https://www.kaggle.com/datasets/crawford/gene-expression/data?select=actual.csv

Pochodzi z badania: https://www.dkfz.de/genomics-proteomics/fileadmin/downloads/Expression/Golub_1999.pdf

The dataset employed in this research was sourced from bone marrow samples obtained from patients diagnosed with acute leukemia. It included samples of acute lymphoblastic leukemia (ALL) and acute myeloid leukemia (AML).

This dataset served as the basis for developing a class predictor to classify new, unknown samples of acute leukemias based on their gene expression profiles.

##Konfiguracja środowiska

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

##Dataset

In [3]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [5]:
!kaggle datasets download -d crawford/gene-expression
!unzip "gene-expression.zip"

Dataset URL: https://www.kaggle.com/datasets/crawford/gene-expression
License(s): CC0-1.0
gene-expression.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  gene-expression.zip
  inflating: actual.csv              
  inflating: data_set_ALL_AML_independent.csv  
  inflating: data_set_ALL_AML_train.csv  


In [7]:
train_df = pd.read_csv('data_set_ALL_AML_train.csv')
test_df = pd.read_csv('data_set_ALL_AML_independent.csv')
patient_result_df = pd.read_csv('actual.csv')

###Opis danych


In [14]:
patient_result_df['cancer'].value_counts()

cancer
ALL    47
AML    25
Name: count, dtype: int64

In the training and testing sets there are 72 patients. Each patient is marked either as "ALL" or "AML" based on the type of leukemia they are diagnosed with.

##Preprocessing

##Selekcja istotnych genów